In [1]:
# !pip install textblob ipywidgets --quiet
from textblob import TextBlob
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time

df_genres = pd.read_csv('data_by_genres.csv')
df_tracks = pd.read_csv('data_w_genres.csv')

mood_emoji = {
    "happy": "😊",
    "sad": "😢",
    "neutral": "😐",
    "angry": "😠",
    "excited": "🤩",
    "relaxed": "😌",
    "invalid": "⚠️"
}

mood_messages = {
    "happy": "You’re radiating good vibes! Time to dance it out.",
    "sad": "It’s okay to feel low. Let the blues soothe you.",
    "neutral": "Balanced and chill — perfect for Lo-Fi beats.",
    "angry": "Turn that fire into power — hard beats ahead!",
    "excited": "You’re buzzing with energy! Let’s go EDM!",
    "relaxed": "Breathe in, breathe out... smooth sounds coming.",
    "invalid": "Please enter something to analyze."
}

def get_genres_for_mood(mood, df):
    if mood == "happy":
        return df[(df['valence'] > 0.6) & (df['energy'] > 0.6)]['genres'].head(5).tolist()
    elif mood == "sad":
        return df[(df['valence'] < 0.4) & (df['energy'] < 0.4)]['genres'].head(5).tolist()
    elif mood == "relaxed":
        return df[(df['valence'] > 0.4) & (df['valence'] < 0.7) & (df['energy'] < 0.5)]['genres'].head(5).tolist()
    elif mood == "angry":
        return df[(df['valence'] < 0.3) & (df['energy'] > 0.7)]['genres'].head(5).tolist()
    elif mood == "excited":
        return df[(df['valence'] > 0.7) & (df['energy'] > 0.7)]['genres'].head(5).tolist()
    elif mood == "neutral":
        return df[(df['valence'] > 0.45) & (df['valence'] < 0.55)]['genres'].head(5).tolist()
    else:
        return ["Unknown Genre"]

def analyze_mood(text):
    text = text.strip()
    if not text:
        return "invalid", "N/A", "Please enter a valid mood description.", mood_emoji["invalid"]

    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    lower_text = text.lower()


    if "furious" in lower_text or "angry" in lower_text or "hate" in lower_text:
        mood = "angry"
    elif "relaxed" in lower_text or "calm" in lower_text:
        mood = "relaxed"
    elif "excited" in lower_text or "pumped" in lower_text or "thrilled" in lower_text:
        mood = "excited"
    elif "sad" in lower_text or "depressed" in lower_text or "down" in lower_text:
        mood = "sad"
    elif "happy" in lower_text or "glad" in lower_text or "great" in lower_text:
        mood = "happy"
    else:
        if polarity >= 0.6:
            mood = "excited"
        elif polarity >= 0.3:
            mood = "happy"
        elif polarity > -0.3:
            mood = "neutral"
        elif polarity >= -0.6:
            mood = "sad"
        else:
            mood = "angry"

    genres = get_genres_for_mood(mood, df_genres)
    emoji = mood_emoji.get(mood, "❓")
    message = mood_messages.get(mood, "")

    explanation = f"""
    <div style='font-family:sans-serif; font-size:16px; padding:12px; background: linear-gradient(to right, #dbeafe, #e0f7fa); border-left: 6px solid #3b82f6; border-radius: 8px;'>
        <h3 style='margin:0;'>🎯 Detected Mood: <span style='color:#2563eb;'>{mood.upper()} {emoji}</span></h3>
        <p><b>📊 Polarity Score:</b> {polarity:.2f}</p>
        <p><b>🎧 Suggested Genres:</b> {', '.join(genres)}</p>
        <p><i>🧠 {message}</i></p>
    </div>
    """
    return explanation

input_box = widgets.Text(
    placeholder="Tell me how you feel...",
    description="Mood:",
    layout=widgets.Layout(width='100%')
)

sample_dropdown = widgets.Dropdown(
    options=[
        ("Try a sample...", ""),
        ("I'm feeling great today!", "I'm feeling great today!"),
        ("I'm feeling really down.", "I'm feeling really down."),
        ("I'm thrilled for this party!", "I'm thrilled for this party!"),
        ("I'm super mad at everything!", "I'm super mad at everything!"),
        ("Just chilling today.", "Just chilling today.")
    ],
    description='Examples:'
)

analyze_button = widgets.Button(description="🎧 Recommend", button_style='success')
clear_button = widgets.Button(description="🧹 Clear", button_style='danger')
live_toggle = widgets.Checkbox(value=False, description="Live Re-analysis 🔄")
output_box = widgets.Output()

def display_result():
    with output_box:
        clear_output(wait=True)
        text = input_box.value
        html_output = analyze_mood(text)
        time.sleep(0.2)
        display(HTML(html_output))

def on_analyze_click(b):
    display_result()

def on_clear_click(b):
    input_box.value = ""
    sample_dropdown.value = ""
    with output_box:
        clear_output()

def on_dropdown_change(change):
    if change["new"]:
        input_box.value = change["new"]
        if live_toggle.value:
            display_result()

def on_input_change(change):
    if live_toggle.value:
        display_result()

analyze_button.on_click(on_analyze_click)
clear_button.on_click(on_clear_click)
sample_dropdown.observe(on_dropdown_change, names='value')
input_box.observe(on_input_change, names='value')

ui = widgets.VBox([
    widgets.HTML("<h2 style='color:#333;'>🎶 Mood-Based Music Recommender (Data-Driven) 🎶</h2>"),
    input_box,
    widgets.HBox([sample_dropdown, live_toggle]),
    widgets.HBox([analyze_button, clear_button]),
    output_box
])

display(ui)